<a href="https://colab.research.google.com/github/jikerbug/jibaek_project_generator/blob/master/yousictube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#yousictube 웹앱 개발일지 1주차

In [ ]:
# client

import requests
from PIL import Image, ImageDraw, ImageFont
from numpy import format_parser

URL = "http://127.0.0.1:5000/predict"
TEST_AUDIO_FILE_PATH = "test/target_music.mp4"



if __name__ == "__main__":

    audio_file = open(TEST_AUDIO_FILE_PATH, "rb")
    values = {"file": (TEST_AUDIO_FILE_PATH, audio_file, "audio/mp4")}
    response = requests.post(URL, files=values)
    data = response.json()

    img = Image.new(mode='RGB', size=(1000,2200), color='#FFF')
    draw = ImageDraw.Draw(img)

    font = ImageFont.truetype('C:/Windows/Fonts/Arial.ttf', 55)



    beat_flag = 0

    text = ''

    for chord in data['chord_list']:
        if float(chord['timestamp']) >= 0 and float(chord['timestamp']) < float(data['beat_list'][0]):
            if chord['label'] == 'N':
                continue
            text += chord['label'] + '  '

    init_flag = True
    for beat_id in range(len(data['beat_list'])):
        if init_flag == False:
            if beat_flag != 16:
                beat_flag += 1
                continue
        else:
            if beat_flag != 10:
                beat_flag += 1
                continue
            init_flag = False

        for chord in data['chord_list']:
            if float(chord['timestamp']) >= float(data['beat_list'][beat_id-beat_flag]) and float(chord['timestamp']) < float(data['beat_list'][beat_id]):
                text += chord['label'] + '  '
        text += '\n\n'
        beat_flag = 0



    draw.text((50, 50), text, font=font, fill='#000')
    img.save('draw_text.png', format='PNG')

    print(f"Predicted chord list is: {data['chord_list']}")
    print(f"Predicted beat list is: {data['beat_list']}")


In [ ]:
#server

# server
# client -> POST request -> server -> prediction back to client
import random
import os
from flask import Flask, request, jsonify
from chord_classification_service import Chord_Classification_Service

app = Flask(__name__)


# ex) Ks.com/predict


@app.route("/predict", methods=["POST"])
def predict():

    # get audio file and save it
    audio_file = request.files["file"]
    file_name = str(random.randint(0, 1000000))
    audio_file.save(file_name)

    # invoke keyword spotting service
    ccs = Chord_Classification_Service()


    # make a prediction
    predicted_chord_list, predicted_beat_list = ccs.predict(file_name)

    chord_list = []
    for i in predicted_chord_list:
        i['timestamp'] = str(i['timestamp'])
        chord_list.append(i)


    beat_list = []
    for i in predicted_beat_list:
        beat_list.append(str(i['timestamp']))

    # remove the audio file
    os.remove(file_name)

    # send back the predicted keyword in json format
    data = {"chord_list" : chord_list, "beat_list" : beat_list}

    return jsonify(data)

if __name__ == "__main__":
    app.run(debug=False)




In [ ]:
# chord_classification_service

import librosa
import vamp


from pytube import YouTube


class _Chord_Classification_Service:

    _instance = None

    def predict(self, file_path):


        # load audio file
        signal, sr = librosa.load(file_path)

        # extract chromagram
        print(vamp.list_plugins())

        beat = vamp.collect(signal, sr, "beatroot-vamp:beatroot")
        print(beat)

        for i in range(len(beat['list']) -1):
            print(beat['list'][i+1]['timestamp'] - beat['list'][i]['timestamp'])


        predicted_list = vamp.collect(signal, sr, "nnls-chroma:chordino")
        print(predicted_list)


        return predicted_list['list'], beat['list']





def  Chord_Classification_Service():

    #ensure that we only have 1 instance of KSS
    if  _Chord_Classification_Service._instance is None:
        _Chord_Classification_Service._instance = _Chord_Classification_Service()
    return  _Chord_Classification_Service._instance

if __name__ == "__main__":

    ccs = Chord_Classification_Service()

    #yt = YouTube('https://www.youtube.com/watch?v=QDYfEBY9NM4').streams[0].download("test", "target_music")

    chord_list = ccs.predict("test/target_music.mp4")
    for i in chord_list:
        print(i)


